# Data Aggregation and Group Operations

In [ ]:
import pandas as pd
import numpy as np

## GroupBy mechanics

In [ ]:
nrows = 10

df = pd.DataFrame({'company' : np.random.choice(list('ab'), nrows),
                   'data1' : np.random.randn(nrows) * 50 + 100,
                   'city' : np.random.choice(list('MP'), nrows),
                   'income' : np.random.randn(nrows) * 30000 + 50000 })

df

In [ ]:
grouped = df.groupby('company')
grouped

In [ ]:
grouped.sum()

In [ ]:
#si solo quiero que me sume una columna
grouped.sum()['income']

In [ ]:
grouped.mean()

In [ ]:
df_gruped_mean = grouped.mean()
df_gruped_mean.index

In [ ]:
# si quiero recuperar la compañia como indice
df_gruped_mean.reset_index()

In [ ]:
df.groupby(['company', 'city']).mean()

In [ ]:
#ejercicio:añadir una columna en df con el total income por ciudad
#1-obtener total income por ciudad
df.groupby('city')['income'].sum() #-> pero esto es una serie, mejor lo pasamos a un dataframe
df_total_income_city = df.groupby('city')['income'].sum().reset_index() # asi tenemos un dataframe, que es mas facil trabajar con el

#2-ahora tenemos que añadirle la info que queremos es decir para cada una de las ciudades el total income
df.merge(df_total_income_city, left_on='city', right_on='city')#pero asi me cambia el nombre de la columna
df.merge(df_total_income_city, left_on='city', right_on='city', suffixes=['','_total_city'])
#y para ordenarlo:
df.merge(df_total_income_city, left_on='city', right_on='city', suffixes=['','_total_city']).sort_index(axis=1)

In [ ]:
# ahora el mismo ejer pero añadiendo por ciudad y company la media(mean)

In [ ]:
agg = df.groupby(['city','company'])['income'].mean().reset_index()#importantisimo el reset index cuando agrupamos por mas de una columna

df_final = df.merge(agg,left_on=['city','company'], right_on=['city','company'], suffixes=['','_mean']).sort_index(axis=1)

df_final

In [ ]:
#ahora añadir si el valor de esa fila esta por encima o no de la media

In [ ]:
df.groupby(['company', 'city']).mean()['income']

In [ ]:
df.groupby(['company', 'city'])['income'].mean()

In [ ]:
means = df.groupby(['company', 'city'])['income'].mean()

In [ ]:
means

In [ ]:
means.index

In [ ]:
means['a', 'M']

In [ ]:
means.reset_index()

### Iterating over groups

In [ ]:
for name, group in df.groupby('company'):
    print(group)

In [ ]:
for name, group in df.groupby(['company','city']):
    print(name)

In [ ]:
all_data = dict(list(df.groupby('company')))
all_data['a']

### Selecting a column or subset of columns

In [ ]:
df

In [ ]:
df.groupby('company')[['data1', 'city']].max()

In [ ]:
df[df['company']=='a'][['data1', 'city']].max()

In [ ]:
df[df['company']=='b'][['data1', 'city']].max()

In [ ]:
df[df['company']=='b'][['data1', 'city']].max().reset_index()

## Data aggregation

In [ ]:
df.groupby('city').median()

In [ ]:
df.groupby('city').quantile(.9)

In [ ]:
stats = df.groupby('city').describe()
stats

In [ ]:
stats['data1','mean']

In [ ]:
stats.columns

In [ ]:
!wget https://raw.githubusercontent.com/wesm/pydata-book/1st-edition/ch08/tips.csv

In [ ]:
tips = pd.read_csv('tips.csv')
tips.head()

In [ ]:
#opciones de day y time en el csv
tips['day'].unique()

In [ ]:
tips['time'].unique()

In [ ]:
tips.describe()

In [ ]:
#calculamos el porcentaje de propina que se ha dejado
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.head()

In [ ]:
#ahora queremos saber si la propina por persona ha sido por debajo de la media o no

In [ ]:
tips['tip'].mean()

In [ ]:
tips['tip_pct_pers'] = (tips['tip'] / tips['total_bill']) / tips['size']
tips.head()

In [ ]:
tips.groupby('time')[['total_bill','tip','tip_pct','size']].agg(['min','max','mean','std'])

In [ ]:
l_stats = ['min','max','mean','std','median']

In [ ]:
tips.groupby('time')[['total_bill','tip','tip_pct','size']].agg(l_stats)

In [ ]:
#o si lo vemos mejor con la traspuesta
tips.groupby('time')[['total_bill','tip','tip_pct','size']].agg(l_stats).T

### Column-wise and multiple function application

In [ ]:
tips.groupby('sex').mean()

In [ ]:
tips.groupby('sex').std()

In [ ]:
tips.groupby('sex').agg(['mean', 'std'])

In [ ]:
tips.groupby(['sex', 'smoker']).agg(['mean', 'std', 'count'])

In [ ]:
tips.groupby(['sex', 'smoker']).agg([np.mean, np.std, np.count_nonzero])

In [ ]:
def minimax(series):
    return series.max() - series.min()

tips.groupby(['sex', 'smoker']).agg([np.mean, np.std, minimax])

In [ ]:
tips.groupby(['sex', 'smoker']).agg({'total_bill' : [np.sum, np.mean], 
                                     'tip' : [minimax, np.std]})

## Group-wise operations and transformations

In [ ]:
tips_by_smoker = tips.groupby('smoker')['tip_pct'].agg([np.mean, np.std])
tips_by_smoker

In [ ]:
merged = tips.merge(tips_by_smoker, left_on='smoker', right_index=True)
merged.head()

### Apply: General split-apply-combine

In [ ]:
def top(df, n=5, col='tip_pct'):
    return df.sort_values(by=col, ascending=False).head(n)

top(merged)

In [ ]:
merged.groupby('sex').apply(top)

#### Suppressing the group keys

In [ ]:
tips.groupby(['sex', 'smoker'], group_keys=False).apply(top)

### Quantile and bucket analysis

In [ ]:
pd.cut(merged['total_bill'], 5)

In [ ]:
merged.groupby(pd.cut(merged['total_bill'], 5))['tip_pct'].agg([np.mean, np.std])

In [ ]:
merged.groupby(merged['size'] > 2)['tip_pct'].agg([np.mean, np.std])

In [ ]:
merged.groupby(merged['size'] == 2)['tip_pct'].agg([np.mean, np.std])